In [28]:
import time
import os
from sagemaker import get_execution_role, session
import boto3
from datetime import datetime

In [23]:
#set up Boto3

region = boto3.Session().region_name

role = get_execution_role()

sm_client = boto3.client('sagemaker', region_name=region)

In [3]:
#Create model group

model_package_group_name = "retail-product-classifier" + str(round(time.time()))
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "Retail Product Classifier"
}

create_model_pacakge_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
print('ModelPackageGroup Arn : {}'.format(create_model_pacakge_group_response['ModelPackageGroupArn']))

ModelPackageGroup Arn : arn:aws:sagemaker:us-east-1:232188586941:model-package-group/retail-product-classifier1618343076


In [11]:
#Register a model version

modelpackage_inference_specification =  { 
    "InferenceSpecification": {    
      "Containers": [ 
         {  
            "Image": '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.1.3-cpu-py36-ubuntu18.04',
            "ModelDataUrl": 's3://sagemaker-us-east-1-232188586941/tensorflow-training-2021-03-17-01-56-43-365/output/model.tar.gz' 
         }
      ],
      "SupportedContentTypes": [ "text/csv" ],
      "SupportedResponseMIMETypes": [ "text/csv" ],
   }
 }

model_url='s3://sagemaker-us-east-1-232188586941/tensorflow-training-2021-03-17-01-56-43-365/output/model.tar.gz'

# Specify the model data
modelpackage_inference_specification["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]=model_url

create_model_package_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageDescription" : "Model to categorize rS products using image and text (title and description)",
    "ModelApprovalStatus" : "PendingManualApproval"
}
create_model_package_input_dict.update(modelpackage_inference_specification)

In [12]:
create_mode_package_response = sm_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_mode_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))

ModelPackage Version ARN : arn:aws:sagemaker:us-east-1:232188586941:model-package/retail-product-classifier1618343076/1


In [13]:
# View model group

sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'retail-product-classifier1618343076',
   'ModelPackageVersion': 1,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:232188586941:model-package/retail-product-classifier1618343076/1',
   'ModelPackageDescription': 'Model to categorize rS products using image and text (title and description)',
   'CreationTime': datetime.datetime(2021, 4, 13, 19, 56, 44, 972000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'}],
 'ResponseMetadata': {'RequestId': 'cc5f60fd-b476-4a5f-947d-9357f83d1460',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cc5f60fd-b476-4a5f-947d-9357f83d1460',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '446',
   'date': 'Tue, 13 Apr 2021 19:58:28 GMT'},
  'RetryAttempts': 0}}

In [14]:
sm_client.describe_model_package(ModelPackageName="arn:aws:sagemaker:us-east-1:232188586941:model-package/retail-product-classifier1618343076/1")

{'ModelPackageGroupName': 'retail-product-classifier1618343076',
 'ModelPackageVersion': 1,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:232188586941:model-package/retail-product-classifier1618343076/1',
 'ModelPackageDescription': 'Model to categorize rS products using image and text (title and description)',
 'CreationTime': datetime.datetime(2021, 4, 13, 19, 56, 44, 972000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.1.3-cpu-py36-ubuntu18.04',
    'ImageDigest': 'sha256:74c95a09fe6618e38ce2257b0779c6327e7b3de5527c03b2757e6c4aaa6c2c23',
    'ModelDataUrl': 's3://sagemaker-us-east-1-232188586941/tensorflow-training-2021-03-17-01-56-43-365/output/model.tar.gz'}],
  'SupportedContentTypes': ['text/csv'],
  'SupportedResponseMIMETypes': ['text/csv']},
 'ModelPackageStatus': 'Completed',
 'ModelPackageStatusDetails': {'ValidationStatuses': [],
  'ImageScanStatuses': []},
 'CertifyForMarketp

In [35]:
model_version_arn="arn:aws:sagemaker:us-east-1:232188586941:model-package/retail-product-classifier1618343076/1"

model_name = 'TEST-modelregistry-model-1'
print("Model name : {}".format(model_name))
primary_container = {
    'ModelPackageName': model_version_arn,
}
create_model_respose = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container
)
print("Model arn : {}".format(create_model_respose["ModelArn"]))

Model name : TEST-modelregistry-model-1
Model arn : arn:aws:sagemaker:us-east-1:232188586941:model/test-modelregistry-model-1


In [37]:
endpoint_config_name = 'DEMO-modelregistry-EndpointConfig-1'
print(endpoint_config_name)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.p2.xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

DEMO-modelregistry-EndpointConfig-1


In [38]:
endpoint_name = 'DEMO-modelregistry-endpoint-1'
print("EndpointName={}".format(endpoint_name))

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

EndpointName=DEMO-modelregistry-endpoint-1
arn:aws:sagemaker:us-east-1:232188586941:endpoint/demo-modelregistry-endpoint-1


In [40]:
sm_client.delete_endpoint(EndpointName = endpoint_name)

ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Cannot update in-progress endpoint "arn:aws:sagemaker:us-east-1:232188586941:endpoint/demo-modelregistry-endpoint-1".